# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [92]:
# import xlwings as xw
# from xlwings.constants import AutoFillType
# import numpy as np
# wb = xw.Book('./07_excel_data/себестоимостьА_в1.xlsx')
# sheet = wb.sheets["Рецептура"]




In [93]:
# cons = sheet.range("G7:O10").options(np.array).value
# cons

In [94]:
# unit = sheet.range("G14:O14").options(np.array).value
# unit

In [95]:
# cost_price = np.nan_to_num(cons * unit).sum(axis=1)
# cost_price

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [96]:
# sheet.range('T7:T10').options(transpose=True).value = cost_price

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [97]:
# sheet.range("T6").value = "Себестоимость" # значение в позиции T6

In [98]:
# sheet.range("T4:T6").api.merge()
# sheet.range("T7:T10").color = (255, 0, 0)

In [99]:
# sheet.range("V7").formula = "=SUMPRODUCT(G7:O7, $G$14:$O$14)"


4. Выполнить 3 с помощью "протягиваемых" формул.

In [100]:
# sheet.range("V7").api.autofill(destination=sheet.range("V7:V10").api,
#                                type=AutoFillType.xlFillDefault)


## Лабораторная работа 7

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [101]:
import xlwings as xw
import pandas as pd
import numpy as np
from xlwings.constants import AutoFillType
# TODO add data files recipes_sample and ...
recipes = pd.read_csv("./07_excel_data/recipes_sample.csv", sep=",", parse_dates=['submitted'])
recipes = recipes.drop(columns=["n_steps", "contributor_id"])
recipes
# 

,name,id,minutes,submitted,description,n_ingredients
0,george s at the cove black bean soup,44123,90,2002-10-25,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,2003-07-26,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,2002-08-29,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,2002-07-27,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,2004-02-23,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,2007-11-25,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,2009-08-24,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,2004-11-03,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,2012-08-29,this is a delicious soup that i originally fou...,NaN


In [102]:
RSWTI = pd.read_csv("./07_excel_data/recipes_sample_with_tags_ingredients.csv")
RSWTI = RSWTI.drop(columns=["n_steps", "contributor_id", "tags", "ingredients", "n_tags"])
RSWTI

,name,id,minutes,submitted,description,n_ingredients
0,george s at the cove black bean soup,44123,90,2002-10-25,an original recipe created by chef scott meska...,18
1,healthy for them yogurt popsicles,67664,10,2003-07-26,my children and their friends ask for my homem...,3
2,i can t believe it s spinach,38798,30,2002-08-29,"these were so go, it surprised even me.",8
3,italian gut busters,35173,45,2002-07-27,my sister-in-law made these for us at a family...,9
4,love is in the air beef fondue sauces,84797,25,2004-02-23,i think a fondue is a very romantic casual din...,12
...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,2007-11-25,this is based on a french recipe but i changed...,10
29996,zwetschgenkuchen bavarian plum cake,386977,240,2009-08-24,"this is a traditional fresh plum cake, thought...",11
29997,zwiebelkuchen southwest german onion cake,103312,75,2004-11-03,this is a traditional late summer early fall s...,13
29998,zydeco soup,486161,60,2012-08-29,this is a delicious soup that i originally fou...,22


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [103]:
recipes_5per = recipes.sample(n=int(len(recipes) * 0.05), replace=False, axis=0)
RSWTI_5per = RSWTI.sample(n=int(len(RSWTI) * 0.05), replace=False, axis=0)

recipes_5per.to_excel("./07_excel_data/recipes.xlsx", sheet_name='Рецепты') 
RSWTI_5per.to_excel("./07_excel_data/recipes.xlsx", sheet_name='Отзывы') 


In [104]:
# recipes_5per

In [105]:
# RSWTI_5per

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [106]:
# TODO
recipes_5per["seconds_assign"] = recipes_5per["minutes"] * 60
recipes_5per

,name,id,minutes,submitted,description,n_ingredients,seconds_assign
26213,stuffing breakfast strata,211256,65,2007-02-13,great for thanksgiving weekend! delicious stra...,8.0,3900
1739,baked applesauce like an apple crisp,277813,70,2008-01-09,yum yum yum! like a super easy apple crisp. ...,6.0,4200
13886,homemade ponzu sauce,487741,5,2012-09-28,a decent substitution for the real thing.,6.0,300
13830,homemade almond milk,355388,4,2009-02-11,this is another of my mom's recipes. i am not ...,3.0,240
29283,white chocolate mousse torte with oreo cookie ...,380164,45,2009-07-03,i've had this for so long that i can no longer...,8.0,2700
...,...,...,...,...,...,...,...
13360,hash browns chicken soup,87915,50,2004-03-31,i found this recipe in our local monthly elect...,7.0,3000
1549,awesome fried pork chops,38190,65,2002-08-26,my mama used to make these all the time. i hop...,7.0,3900
11465,fried dill pickle coins,96661,40,2004-07-30,this recipe is in the new taste of home magazi...,NaN,2400
13124,guam red rice,241150,30,2007-07-18,being a military family we get the pleasure of...,NaN,1800


In [112]:
wb = xw.Book('./07_excel_data/recipes.xlsx')
sheet = wb.sheets["Рецепты"]
sheet.range("H1").value = "seconds_assign"
# sheet.range("H2:H1501").options(transpose=True).value = list(recipes_5per["seconds_assign"].to_numpy())
# sheet.range("H2:H4").options(transpose=True).value = [1, 2, 3]

# type(recipes_5per["seconds_assign"].to_frame())
sheet.range('H2').value = recipes_5per["seconds_assign"].to_frame()
sheet.range('H2:H1501').options(pd.DataFrame).value

CommandError: Command failed:
		OSERROR: -1728
		MESSAGE: The object you are trying to access does not exist
		COMMAND: app(pid=370).workbooks['recipes.xlsx'].worksheets['Рецепты'].name.get()

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

5. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

6. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

7. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом